## Word Embeddings

![替代文字](http://kylinhub.oss-cn-shanghai.aliyuncs.com/2019-11-04-%E6%88%AA%E5%B1%8F2019-11-04%E4%B8%8B%E5%8D%884.08.54.png)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2words in vocab,5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix['hello']], dtype=torch.long)
hello_embeds = embeds(lookup_tensor)
print(hello_embeds)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


## N-Gram

![替代文字](http://kylinhub.oss-cn-shanghai.aliyuncs.com/2019-11-04-%E6%88%AA%E5%B1%8F2019-11-04%E4%B8%8B%E5%8D%884.09.03.png)

In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# build a list of tuples. Each tuple is ([word_i-2, word_i-1], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
 
print(trigrams[:3])
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}
print(word_to_ix)
 
 
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
 
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
 
 
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)
 
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss
    losses.append(total_loss)
print(losses)

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
{'by': 0, 'Then': 1, 'To': 2, 'much': 3, 'his': 4, "totter'd": 5, 'thine!': 6, 'deep': 7, 'shame,': 8, 'brow,': 9, 'held:': 10, 'Where': 11, 'days;': 12, "'This": 13, 'the': 14, 'beauty': 15, 'a': 16, 'proud': 17, 'This': 18, 'Will': 19, 'when': 20, 'be': 21, 'eyes,': 22, 'made': 23, 'forty': 24, 'so': 25, 'warm': 26, 'praise': 27, 'to': 28, 'succession': 29, 'trenches': 30, 'couldst': 31, 'Shall': 32, 'fair': 33, 'weed': 34, 'and': 35, 'on': 36, 'Were': 37, 'answer': 38, 'sum': 39, 'dig': 40, 'use,': 41, 'lusty': 42, 'cold.': 43, 'mine': 44, 'say,': 45, 'own': 46, 'see': 47, 'thou': 48, 'When': 49, 'worth': 50, 'child': 51, 'where': 52, 'lies,': 53, "feel'st": 54, 'gazed': 55, 'How': 56, 'winters': 57, 'thriftless': 58, 'shall': 59, 'make': 60, 'field,': 61, 'asked,': 62, 'now,': 63, 'praise.': 64, "beauty's": 65, 'new': 66, 'blood': 67, 'within': 68, "youth's": 69, 'of': 70, 'livery':

## Continuous Bag-of-Words

![替代文字](http://kylinhub.oss-cn-shanghai.aliyuncs.com/2019-11-04-%E6%88%AA%E5%B1%8F2019-11-04%E4%B8%8B%E5%8D%884.09.18.png)

In [5]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM=20
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()
 
# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
 
word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])
 
 
class CBOW(nn.Module):
 
    def __init__(self,vocab_size,context_size,embedding_dim):
        super(CBOW,self).__init__()
        self.embeddings=nn.Embedding(vocab_size,embedding_dim)
        self.linear1=nn.Linear(context_size*2*embedding_dim,256)
        self.linear2=nn.Linear(256,vocab_size)
 
 
    def forward(self, inputs):
        embeds=self.embeddings(inputs).view(1,-1)
        out=F.relu(self.linear1(embeds))
        out=self.linear2(out)
        out=F.log_softmax(out,dim=1)
        return out
 
 
# create your model and train.  here are some functions to help you make
# the data ready for use by your module
 
 
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)
 
 
make_context_vector(data[0][0], word_to_ix)  # example
losses=[]
loss_function=nn.NLLLoss()
model=CBOW(vocab_size,CONTEXT_SIZE,EMBEDDING_DIM)
optimizer=optim.SGD(model.parameters(),lr=0.001)
 
for epoch in range(10):
    total_loss=0
    for context,target in data:
        context_idxs=make_context_vector(context,word_to_ix)
        model.zero_grad()
        log_probs1=model(context_idxs)
        loss=loss_function(log_probs1,torch.tensor([word_to_ix[target]],dtype=torch.long))
        total_loss += loss
        loss.backward()
        optimizer.step()
    losses.append(total_loss)
 
print(losses)

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
[tensor(227.9538, grad_fn=<AddBackward0>), tensor(224.9767, grad_fn=<AddBackward0>), tensor(222.0318, grad_fn=<AddBackward0>), tensor(219.1151, grad_fn=<AddBackward0>), tensor(216.2250, grad_fn=<AddBackward0>), tensor(213.3595, grad_fn=<AddBackward0>), tensor(210.5185, grad_fn=<AddBackward0>), tensor(207.6991, grad_fn=<AddBackward0>), tensor(204.8991, grad_fn=<AddBackward0>), tensor(202.1195, grad_fn=<AddBackward0>)]


## 序列模型和LSTM（Long-Short Term Memory）网络

![替代文字](http://kylinhub.oss-cn-shanghai.aliyuncs.com/2019-11-06-%E6%88%AA%E5%B1%8F2019-11-06%E4%B8%8B%E5%8D%883.21.35.png)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
torch.manual_seed(1)

In [7]:
lstm=nn.LSTM(3,4)#Iput dim is 3, output dim is 4
inputs=[torch.randn(1,3) for _ in range(5)]#make a sequence of length
 
#initialize the hidden state
hidden=(torch.randn(1,1,4),torch.randn(1,1,4))
 
 
for i in inputs:
    out,hidden=lstm(i.view(1,1,-1),hidden)
 
inputs=torch.cat(inputs).view(len(inputs),1,-1)
hidden=(torch.randn(1,1,4),torch.randn(1,1,4))
out,hidden=lstm(inputs,hidden)
print(out)
print(hidden)

tensor([[[-0.2566,  0.1294,  0.0441, -0.5235]],

        [[-0.4444,  0.0762,  0.0304, -0.3889]],

        [[-0.1741,  0.1061, -0.0179, -0.1505]],

        [[-0.1409,  0.1110,  0.0773, -0.2373]],

        [[-0.2308,  0.1164, -0.0115, -0.2423]]], grad_fn=<StackBackward>)
(tensor([[[-0.2308,  0.1164, -0.0115, -0.2423]]], grad_fn=<StackBackward>), tensor([[[-0.4093,  0.6065, -0.0288, -0.4107]]], grad_fn=<StackBackward>))


## 使用LSTM实现词性标注

![替代文字](http://kylinhub.oss-cn-shanghai.aliyuncs.com/2019-11-06-%E6%88%AA%E5%B1%8F2019-11-06%E4%B8%8B%E5%8D%883.24.01.png)

In [15]:
def prepare_sequence(seq,to_ix):
    idxs=[to_ix[w] for w in seq]
    return torch.tensor(idxs,dtype=torch.long)
 
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

test_data = [
             ("I Love You".split()),
             ("I Miss you today".split()),
             ("I May fall in love with Miss Zhou".split())]

word_to_ix={}
for sent,tag in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word]=len(word_to_ix)
print(word_to_ix)
 
tag_to_ix={"DET":0,"NN":1,"V":2}
EMBEDDING_DIM=6
HIDDEN_DIM=6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [0]:
class LSTMTagger(nn.Module):
    def __init__(self,embedding_dim,hidden_dim,vocab_size,tagset_size):
        super(LSTMTagger,self).__init__()
        self.hidden_dim=hidden_dim
        self.word_embeddings=nn.Embedding(vocab_size,embedding_dim)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim)
        self.hidden2tag=nn.Linear(hidden_dim,tagset_size)
        self.hidden=self.init_hidden()
    def init_hidden(self):
        #the axes semantics are (num_layers,minibatch_size,hidden_size)
        return (torch.zeros(1,1,self.hidden_dim),
                (torch.zeros(1,1,self.hidden_dim)))
    def forward(self, sentence):
        embeds=self.word_embeddings(sentence)
        lstm_out,self.hidden=self.lstm(embeds.view(len(sentence),1,-1),self.hidden)
        tag_space=self.hidden2tag(lstm_out.view(len(sentence),-1))
        tag_scores=F.log_softmax(tag_space,dim=1)
        return tag_scores

In [18]:
model=LSTMTagger(EMBEDDING_DIM,HIDDEN_DIM,len(word_to_ix),len(tag_to_ix))
loss_function=nn.NLLLoss()
optimizer=optim.SGD(model.parameters(),lr=0.1)
#before training
with torch.no_grad():
    inputs=prepare_sequence(training_data[0][0],word_to_ix)
    tag_scores=model(inputs)
    print(tag_scores)
 
for epoch in range(300):
    for sentence,tags in training_data:
        model.zero_grad()
        model.hidden=model.init_hidden()
        sentence_in=prepare_sequence(sentence,word_to_ix)
        targets=prepare_sequence(tags,tag_to_ix)
        tag_scores=model(sentence_in)
        loss=loss_function(tag_scores,targets)
        loss.backward()
        optimizer.step()
# after training
with torch.no_grad():
    inputs=prepare_sequence(training_data[0][0],word_to_ix)
    tag_scores=model(inputs)
    print(tag_scores)

tensor([[-1.1960, -0.8261, -1.3477],
        [-1.1106, -0.8343, -1.4421],
        [-1.0676, -0.8322, -1.5093],
        [-1.1764, -0.8032, -1.4118],
        [-0.9803, -0.9185, -1.4887]])
tensor([[-0.0596, -2.9237, -5.4934],
        [-4.0336, -0.0288, -4.5418],
        [-3.8808, -3.7379, -0.0455],
        [-0.0210, -4.2153, -5.1090],
        [-5.2799, -0.0085, -5.7061]])
